In [1]:
import numpy as np
import plotly.graph_objects as go
import random

In [2]:
import random
from typing import List, Tuple
import plotly.graph_objects as go
import numpy as np

class MapVisualizer:
    def __init__(self, dimensions: int, bounds: Tuple[int, int, int], map_type="multi_obstacle_narrow_passage"):
        self.dimensions = dimensions
        self.bounds = bounds
        self.map_type = map_type
        self.data = []
        self.shapes = []
        self.obstacles = []  # List of tuples: 2D (x0,y0,x1,y1), 3D (x0,y0,z0,x1,y1,z1)
        self._generate()

    def _generate(self):
        if self.dimensions == 2:
            self._generate_2d_map()
        elif self.dimensions == 3:
            self._generate_3d_map()
        else:
            raise ValueError("Only 2D and 3D supported.")

    def _check_overlap_2d(self, new_rect):
        for ox0, oy0, ox1, oy1 in self.obstacles:
            if not (new_rect[2] <= ox0 or new_rect[0] >= ox1 or new_rect[3] <= oy0 or new_rect[1] >= oy1):
                return True  # Overlap
        return False

    def _generate_2d_map(self):
        W, H = self.bounds[:2]
        attempts = 0
        while len(self.obstacles) < 10 and attempts < 1000:
            x0 = random.randint(0, W - 10)
            y0 = random.randint(0, H - 10)
            x1 = x0 + random.randint(4, 8)
            y1 = y0 + random.randint(4, 8)
            rect = (x0, y0, x1, y1)
            if not self._check_overlap_2d(rect):
                self.obstacles.append(rect)
            attempts += 1

        for x0, y0, x1, y1 in self.obstacles:
            self.shapes.append(dict(
                type='rect',
                x0=x0, y0=y0,
                x1=x1, y1=y1,
                line=dict(color='purple'),
                fillcolor='purple',
                opacity=0.7,
            ))

    def _check_overlap_3d(self, new_box):
        for ox0, oy0, oz0, ox1, oy1, oz1 in self.obstacles:
            if not (new_box[3] <= ox0 or new_box[0] >= ox1 or
                    new_box[4] <= oy0 or new_box[1] >= oy1 or
                    new_box[5] <= oz0 or new_box[2] >= oz1):
                return True
        return False

    def _generate_3d_map(self):
        W, H, D = self.bounds
        attempts = 0
        while len(self.obstacles) < 20 and attempts < 1000:
            x0, y0, z0 = random.randint(0, W - 10), random.randint(0, H - 10), random.randint(0, D - 10)
            dx, dy, dz = random.randint(3, 6), random.randint(3, 6), random.randint(3, 6)
            box = (x0, y0, z0, x0 + dx, y0 + dy, z0 + dz)
            if not self._check_overlap_3d(box):
                self.obstacles.append(box)
            attempts += 1

        for x0, y0, z0, x1, y1, z1 in self.obstacles:
            self.data.append(go.Mesh3d(
                x=[x0, x1, x1, x0, x0, x1, x1, x0],
                y=[y0, y0, y1, y1, y0, y0, y1, y1],
                z=[z0, z0, z0, z0, z1, z1, z1, z1],
                i=[0, 0, 0, 1, 1, 2, 2, 3, 4, 4, 5, 6],
                j=[1, 2, 3, 2, 5, 3, 6, 0, 5, 6, 6, 7],
                k=[2, 3, 0, 5, 6, 6, 7, 4, 6, 7, 4, 4],
                color='purple',
                opacity=0.7
            ))

    def plot_start_goal(self, start: Tuple, goal: Tuple):
        if self.dimensions == 2:
            self.data += [
                go.Scatter(x=[start[0]], y=[start[1]], mode='markers', marker=dict(size=10, color='orange'), name="Start"),
                go.Scatter(x=[goal[0]], y=[goal[1]], mode='markers', marker=dict(size=10, color='green'), name="Goal")
            ]
        else:
            self.data += [
                go.Scatter3d(x=[start[0]], y=[start[1]], z=[start[2]], mode='markers',
                             marker=dict(size=5, color='orange'), name="Start"),
                go.Scatter3d(x=[goal[0]], y=[goal[1]], z=[goal[2]], mode='markers',
                             marker=dict(size=5, color='green'), name="Goal")
            ]

    def plot_path_trace(self, trace: List[Tuple]):
        if not trace:
            return  # do nothing
        if self.dimensions == 2:
            x, y = zip(*trace)
            self.data.append(go.Scatter(x=x, y=y, mode='lines+markers', line=dict(color='red', width=3), name="Path"))
        else:
            x, y, z = zip(*trace)
            self.data.append(go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                                          line=dict(color='red', width=4), name="Path"))

    def show(self):
        if self.dimensions == 2:
            layout = go.Layout(
                title=f"2D Map - {self.map_type}",
                width=700, height=700,
                xaxis=dict(showgrid=False),
                yaxis=dict(showgrid=False, autorange='reversed'),
                shapes=self.shapes
            )
            fig = go.Figure(data=self.data, layout=layout)
        else:
            fig = go.Figure(data=self.data)
            fig.update_layout(
                title=f"3D Map - {self.map_type}",
                scene=dict(
                    xaxis=dict(nticks=10),
                    yaxis=dict(nticks=10),
                    zaxis=dict(nticks=10)
                ),
                width=700,
                height=700
            )
        fig.show()


In [3]:
vis2d = MapVisualizer(dimensions=2, bounds=(60, 60), map_type="multi_obstacle_narrow_passage")
vis2d.plot_start_goal((5, 5), (55, 55))
vis2d.plot_path_trace([])  # 경로가 없을 경우
vis2d.show()


In [4]:
vis3d = MapVisualizer(dimensions=3, bounds=(50, 50, 30), map_type="cluttered_and_narrow")
vis3d.plot_start_goal((0, 0, 0), (40, 40, 25))
vis3d.plot_path_trace([(0, 0, 0), (10, 10, 10), (20, 20, 15), (40, 40, 25)])
vis3d.show()

In [5]:
import math
class Node:
    def __init__(self, x: int, y: int, parent=None):
        self.x = x
        self.y = y
        self.parent = parent
class RRT:
    def __init__(self, start, goal, bounds, obstacles, max_iter=500, step_size=5):
        self.start = Node(*start)
        self.goal = Node(*goal)
        self.bounds = bounds
        self.obstacles = obstacles  # List of (x0,y0,x1,y1)
        self.max_iter = max_iter
        self.step_size = step_size
        self.nodes = [self.start]
        self.goal_node = None

    def distance(self, n1, n2):
        return math.hypot(n1.x - n2.x, n1.y - n2.y)

    def random_point(self):
        return random.randint(0, self.bounds[0] - 1), random.randint(0, self.bounds[1] - 1)

    def nearest(self, x, y):
        return min(self.nodes, key=lambda node: (node.x - x) ** 2 + (node.y - y) ** 2)

    def steer(self, from_node, to_x, to_y):
        angle = math.atan2(to_y - from_node.y, to_x - from_node.x)
        new_x = int(from_node.x + self.step_size * math.cos(angle))
        new_y = int(from_node.y + self.step_size * math.sin(angle))
        return Node(new_x, new_y, from_node)

    def is_collision_free(self, node):
        x, y = node.x, node.y
        if not (0 <= x < self.bounds[0] and 0 <= y < self.bounds[1]):
            return False
        for ox0, oy0, ox1, oy1 in self.obstacles:
            if ox0 <= x <= ox1 and oy0 <= y <= oy1:
                return False
        return True

    def extract_path(self, node):
        path = []
        while node:
            path.append((node.x, node.y))
            node = node.parent
        return path[::-1]

    def run(self):
        for _ in range(self.max_iter):
            rx, ry = self.random_point()
            nearest_node = self.nearest(rx, ry)
            new_node = self.steer(nearest_node, rx, ry)

            if not self.is_collision_free(new_node):
                continue

            self.nodes.append(new_node)

            if self.distance(new_node, self.goal) < self.step_size:
                self.goal_node = Node(self.goal.x, self.goal.y, new_node)
                self.nodes.append(self.goal_node)
                return self.extract_path(self.goal_node)

        return None


In [10]:
start, goal = (5, 5), (55, 55)
vis = MapVisualizer(dimensions=2, bounds=(60, 60), map_type="multi_obstacle_narrow_passage")
vis.plot_start_goal(start, goal)

# 2. RRT 실행
rrt = RRT(start=start, goal=goal, bounds=(60, 60), obstacles=vis.obstacles)
path = rrt.run()

# 3. 트리 edge 그리기
for node in rrt.nodes:
    if node.parent:
        vis.data.append(
            go.Scatter(x=[node.x, node.parent.x], y=[node.y, node.parent.y],
                        mode='lines', line=dict(color='blue', width=1), showlegend=False)
        )

# 4. 경로 표시
if path:
    vis.plot_path_trace(path)
else:
    print("No path found.")

# 5. 시각화 출력
vis.show()